## Exercise 1 Hello World

1. Write an MPI program displaying the number of processes used for the execution and the rank of each process.
2. Test the programs obtained with different numbers of threads for the parallel program.

**Output Example**
```shell
Hello from the rank 2 process
Hello from the rank 0 process
Hello from the rank 3 process
Hello from the rank 1 process
Parallel execution of hello_world with 4 process
```
*Note that the output order maybe different*

In [2]:
 %%file hello.py
 # write your program here

from mpi4py import MPI
comm=MPI.COMM_WORLD()
taille=COMM.Get_size()
rank=COMM.Get_rank()
print(f'Hello from the rank {rank} process')
print(f'Parallel execution of hello_world with {taille} process')

Overwriting hello.py


In [7]:
# enter command for compile and run the program
!mpirun -n 2 python hello.py

Traceback (most recent call last):
  File "hello.py", line 3, in <module>
    from mpi4py import MPI
ModuleNotFoundError: No module named 'mpi4py'
Traceback (most recent call last):
  File "hello.py", line 3, in <module>
    from mpi4py import MPI
ModuleNotFoundError: No module named 'mpi4py'
--------------------------------------------------------------------------
Primary job  terminated normally, but 1 process returned
a non-zero exit code. Per user-direction, the job has been aborted.
--------------------------------------------------------------------------
--------------------------------------------------------------------------
mpirun detected that one or more processes exited with non-zero status, thus causing
the job to be terminated. The first process to do so was:

  Process name: [[6277,1],0]
  Exit code:    1
--------------------------------------------------------------------------


## Exercise 2 Sharing Data 

A common need is for one process to get data from the user, either by reading from the terminal or command line arguments, and then to distribute this information to all other processors.

Write a program that reads an integer value from the terminal and distributes the value to all of the MPI processes. Each process should print out its rank and the value it received. Values should be read until a negative integer is given as input.

You may want to use these MPI routines in your solution:
`Get_rank` `Bcast` 

**Output Example**
```shell
10
Process 0 got 10
Process 1 got 10
```

In [6]:
 %%file sharing.py
 # write your program here

Writing sharing.py


In [ ]:
# enter command for compile and run the program

## Exercise 3 Sending in a ring (broadcast by ring)

Write a program that takes data from process zero and sends it to all of the other processes by sending it in a ring. That is, process i should receive the data and send it to process i+1, until the last process is reached.
Assume that the data consists of a single integer. Process zero reads the data from the user.
![](../data/ring.gif)

You may want to use these MPI routines in your solution:
`Send` `Recv` 

## Exercise 4 Matrix vector product

1. Use the `MatrixVectorMult.py` file to implement the MPI version of matrix vector multiplication.
2. Process 0 compares the result with the `dot` product.
3. Plot the scalability of your implementation. 

**Output Example**
```shell
CPU time of parallel multiplication using 2 processes is  174.923446
The error comparing to the dot product is : 1.4210854715202004e-14
```

In [5]:
 %%file MatrixVectorMult_V0.py
 # write your program here

Writing MatrixVectorMult_V0.py


In [ ]:
# enter command for compile and run the program

## Exercise 5 Calculation of π (Monte Carlo)

1. Use the `PiMonteCarlo.py` file to implement the calculation of PI using Monte Carlo.
2. Process 0 prints the result.
3. Plot the scalability of your implementation. 

In [ ]:
 %%file PiMonteCarlo_V0.py
 # write your program here

In [ ]:
# enter command for compile and run the program